In [48]:
# pip install signalp-6-package/

In [49]:
# pip install tape-proteins

In [50]:
#HuggingFace Tranformers Package
# pip install transformers

In [51]:
# pip install keras

In [79]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from scipy.stats import norm
import scipy.stats as sps
import cv2
from scipy.ndimage import convolve
import skimage
from scipy import ndimage
from scipy.spatial.distance import cdist
from nibabel.testing import data_path
import nibabel as nib
from sklearn.mixture import GaussianMixture
import os
import pandas as pd
import skimage.measure
import seaborn as sns
import scipy as sp
import gudhi as gd
import random
import sys
sys.path.append('../../') 
from Code.function_geometry import * 
from Code.codegeometry import * 
from PIL import Image
from sklearn.utils import shuffle

In [80]:
import sklearn.preprocessing   as skp
import sklearn.neighbors       as skn
import sklearn.model_selection as skm
import sklearn.decomposition   as skd
import sklearn.manifold        as skf
import sklearn.pipeline        as skl
import sklearn.svm             as sks
import sklearn.ensemble        as ske
import itertools
import tensorflow as tf

import gudhi.clustering.tomato as gdt
import gudhi.representations   as gdr

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
#tokenizer
from tape import ProteinBertModel, TAPETokenizer
# from tape import UniRepModel, UniRepTokenizer
# from tape import ProteinBertModel, ProteinBertTokenizer
from transformers import AutoTokenizer
from transformers import BertModel, BertTokenizer
# ProtGPT2
# from transformers import pipeline

In [83]:
from keras.models import Sequential
from keras.layers import Conv1D,Conv2D, Dense, Flatten, Dropout,MaxPooling2D
from keras import layers
from keras.layers import MaxPooling1D,MaxPooling2D
from keras.optimizers import SGD, RMSprop
from keras import regularizers

In [56]:
def readFastaFiles(filename):
    sequences = []
    labels = []
    with open(filename, 'r') as file:
        current_id = None
        current_seq = ''
        for line in file:
            if line.startswith('>'):
                if current_id and len(current_seq) > 0 and len(current_seq) <= 201:
                    labels.append(current_id[1:])
                    sequences.append(current_seq.replace('*', '').replace('\n', ''))
                current_id = line
                current_seq = ''
            else:
                current_seq += line
        # Add the last sequence
        if current_id and len(current_seq) > 0 and len(current_seq) <= 201:
            labels.append(current_id)
            sequences.append(current_seq.replace('*', '').replace('\n', ''))
    return labels,sequences

In [57]:
def createFaster(filename,labels,sequences):
    with open(filename, 'w') as f:
        # Write content to the file
        for i in range(len(sequences)):
        # for i in range(5):
            f.write(labels[i])
            f.write(sequences[i]+"\n")

In [59]:
#Creating a file of all signalP sequences
#getting supposed signalprotein
def signalProtein(filename,resultSignalp,sequences,labels):
    new_label = []
    new_sequence = []
    i = 0
    with open(filename, 'w') as f:
        for r in result_signalp:
            if r > 0 and r < 0.7:
                new_label.append(0)
            elif r > 0.7:
                new_label.append(1)
            if r > 0:
                f.write(labels[i]+"\n")
                f.write(sequences[i]+"\n")
                new_sequence.append(sequences[i])
            i+=1
    return new_label,new_sequence         

In [60]:
def resultSignalP(filename):
    f=open(filename,'r')
    result=[]
    next(f)
    next(f)
    for line in f.readlines():
        data = line.rstrip()  
        index = data.__contains__('SP')
        #Can we return the probabilty of SP and other
        if index == True: 
            result.append(1)
        else:
            result.append(0)
    f.close()
    return result  

In [61]:
def geotop_analysis(data):
    n = 100
    i = 0
    dgms_tda = []
    dgms_per_var = []
    dgms_per_linalg = []
    for U in data:
        print(i)
        L = np.linspace(np.min(U), np.max(U),  n)[::-1]
        life, barecode, persistence, connected_comp, Per_total, Area_total, euler_total, per, area, euler = function_persistance(U, L, False)
        diagram_pondere_total_vari = np.zeros((len(life), 2))
        diagram_pondere_linalg = np.zeros((len(life), 2))
        diagram = np.zeros((len(life), 2))
        k = 0
        idx_keys = list(life.keys())
        for idx in idx_keys[1:]:
            diagram_pondere_total_vari[k][0] = (np.sum(per[idx]))*life[idx][1]
            diagram_pondere_total_vari[k][1] = (np.sum(per[idx]))*life[idx][0]
            diagram_pondere_linalg[k][0] = (np.linalg.norm(per[idx]))*life[idx][1]
            diagram_pondere_linalg[k][1] = (np.linalg.norm(per[idx]))*life[idx][0]
            diagram[0] = life[idx][1]
            diagram[1] = life[idx][0]
            k=k+1
        dgms_tda.append(diagram)
        dgms_per_linalg.append(diagram_pondere_linalg)
        dgms_per_var.append(diagram_pondere_total_vari)
        i=i+1
    return dgms_per_linalg

In [62]:
def concate_embedding_geotop(embedding,dgms,dim):
    long = embedding.shape[1]
    res = []
    i = 0
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate
        t = dim - (long + flat.shape[0])
        res.append(np.concatenate((np.concatenate((embedding[i], flat)), np.zeros(t))))
        i +=1
    return res

In [63]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.15.0
Keras version: 2.15.0


In [64]:
def concate_embedding_geotop(embedding,dgms,dim):
    # len_data = max(array.shape[0] for array in dgms)
    long = embedding.shape[1]
    res = []
    i = 0
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest with zeroes
        t = dim - (long + flat.shape[0])
        res.append(np.concatenate((np.concatenate([embedding[i], flat]), np.zeros(t))))
        i +=1
    return res

In [65]:
def concat_embedding_geotop(dgms,dim):
    # len_data = max(array.shape[0] for array in dgms)
    res = []
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest with zeroes
        t = dim - flat.shape[0]
        res.append(np.concatenate([flat, np.zeros(t)]))
    return res

In [69]:
label,sequence=readFastaFiles("Araport11_pep_20220914")  # change to your own path

In [70]:
len(sequence)

10280

In [71]:
len(label)

10280

In [72]:
np.save("labels.npy",label)

## Data Manipulation

In [73]:
#Creating fasta file with protein <=200
# createFaster("arabidopsis_new.fasta",label,sequence)

### Embedding using TAPE

In [74]:
#Learning the representation of protein
model_tape = ProteinBertModel.from_pretrained('bert-base')
tokenizer_tape = TAPETokenizer(vocab='iupac')

In [76]:
len(label)

10280

In [ ]:
num_of_features = 768
embedded=np.zeros((len(label),num_of_features))
y=np.zeros(len(label))

i=0
for s in sequence:
    #tape
    print(i)
    token_ids = torch.tensor([tokenizer_tape.encode(s)])
    output = model_tape(token_ids)
    sequence_output = output[0]
    embedded[i]= np.array(np.mean(sequence_output.detach().numpy(),axis=1) )
    i=i+1

In [ ]:
np.save('arabidopsis_tape.npy', embedded)

In [ ]:
embedded.shape

(10280, 768)

In [ ]:

import numpy as np
# Save the array to a file
# np.save('arabidopsis_tape.npy', embedded)

# # Later, to reload the array
embedded = np.load('arabidopsis_tape.npy')

In [ ]:
embedded_3d = embedded.reshape(embedded.shape[0],embedded.shape[1],1)

In [ ]:
embedded_3d.shape

(10280, 768, 1)

### Classification

In [ ]:
from gtda.homology import VietorisRipsPersistence

# Track connected components, loops, and voids
homology_dimensions = [0]

# Collapse edges to speed up H2 persistence calculation!
persistence = VietorisRipsPersistence(
    metric="euclidean",
    homology_dimensions=homology_dimensions,
    n_jobs=6,
    collapse_edges=True,
)

persistence_diagrams = persistence.fit_transform(embedded_3d)

In [ ]:
np.save("persitence_data.npy",persistence_diagrams)

In [ ]:
persistence_diagrams = np.load("persitence_data.npy")

In [85]:
from gtda.plotting import plot_diagram
from scipy.stats import wasserstein_distance

# Circle
# plot_diagram(persistence_diagrams[3])

In [ ]:
len(persistence_diagrams[0])

767

In [ ]:
np.save('persistence_diagrams.npy', persistence_diagrams)

In [ ]:
np.save('persistence_data.npy', persistence_diagrams)

In [ ]:
# Compute the Wasserstein distance between the two diagrams
distance = wasserstein_distance(persistence_diagrams[0][:,1], persistence_diagrams[0][:,1])

print("Wasserstein distance:", distance)


Wasserstein distance: 0.0


In [ ]:
length = len(persistence_diagrams)

In [ ]:
sequence[1262]

'MGIQTRSVLIIVAILTMMFSAHIAQSNSITMCVKHCAQNECLKAAKKPTPKICDEACKKICNNQLFGDEKWFVPAPKGSSRICRWAPKYCQF'

In [ ]:
for i in range(len(persistence_diagrams)):
    for j in range(len(persistence_diagrams)):
        wessertein_dist[i,j] = wasserstein_distance(persistence_diagrams[i][:,1], persistence_diagrams[j][:,1])

In [ ]:
length

10280

In [ ]:
wessertein_dist = np.zeros(shape=(length, length))

In [ ]:
persistence_diagrams[0]

array([[0.00000000e+00, 5.06639481e-06, 0.00000000e+00],
       [0.00000000e+00, 5.45382500e-06, 0.00000000e+00],
       [0.00000000e+00, 7.83801079e-06, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.79704452e-01, 0.00000000e+00],
       [0.00000000e+00, 3.19888592e-01, 0.00000000e+00],
       [0.00000000e+00, 7.55657578e+00, 0.00000000e+00]])

In [ ]:
wessertein_dist.shape

(10280, 10280)

In [ ]:
len(label)

10280

In [ ]:
df = pd.DataFrame(wessertein_dist, index=label, columns=label)

In [ ]:
df

,AT1G01020.2 | Symbols: ARV1 | | chr1:7315-8666 REVERSE LENGTH=191\n,AT1G01020.6 | Symbols: ARV1 | | chr1:7315-8419 REVERSE LENGTH=104\n,"AT1G01100.1 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n","AT1G01100.2 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n","AT1G01100.3 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=96\n","AT1G01100.4 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n",AT1G01130.1 | Symbols: no symbol available | no full name available | chr1:61905-63811 REVERSE LENGTH=124\n,AT1G01160.1 | Symbols: GIF2 | GRF1-interacting factor 2 | chr1:72583-73883 FORWARD LENGTH=195\n,AT1G01170.1 | Symbols: no symbol available | no full name available | chr1:74105-74443 REVERSE LENGTH=83\n,AT1G01170.2 | Symbols: no symbol available | no full name available | chr1:74105-74443 REVERSE LENGTH=83\n,...,ATMG00070.1 | Symbols: NAD9 | NADH dehydrogenase subunit 9 | chrm:24665-25237 FORWARD LENGTH=190\n,ATMG00080.1 | Symbols: RPL16 | ribosomal protein L16 | chrm:23285-23824 FORWARD LENGTH=179\n,ATMG00210.1 | Symbols: RPL5 | ribosomal protein L5 | chrm:184762-185319 REVERSE LENGTH=185\n,"ATMG00480.1 | Symbols: ATP8, ORFB | ATP SYNTHASE 8 | chrm:247016-247492 REVERSE LENGTH=158\n",ATMG00640.1 | Symbols: ORF25 | | chrm:362206-362784 FORWARD LENGTH=192\n,ATMG00980.1 | Symbols: RPSL2 | | chrm:288446-288823 FORWARD LENGTH=125\n,ATMG00990.1 | Symbols: NAD3 | NADH dehydrogenase 3 | chrm:288044-288400 FORWARD LENGTH=118\n,ATMG01080.1 | Symbols: ATP9 | mitochondrial F0-ATPase subunit 9 | chrm:270961-271185 REVERSE LENGTH=74\n,ATMG01270.1 | Symbols: RPS7 | mitochondrial ribosomal protein S7 | chrm:79559-80005 FORWARD LENGTH=148\n,"ATMG01275.1 | Symbols: ND1, NAD1, NAD1C, NAD1A | NADH DEHYDROGENASE 1, NADH dehydrogenase 1A, NADH dehydrogenase 1C, NAD(P)H DEHYDROGENASE 1 | chrm:58315-59481 FORWARD LENGTH=90\n"
AT1G01020.2 | Symbols: ARV1 | | chr1:7315-8666 REVERSE LENGTH=191\n,0.000000,0.002341,0.004177,0.004177,0.002867,0.004177,0.003282,0.006169,0.002265,0.002265,...,0.002429,0.003655,0.002427,0.001466,0.002080,0.003497,0.001379,0.000711,0.004055,0.001154
AT1G01020.6 | Symbols: ARV1 | | chr1:7315-8419 REVERSE LENGTH=104\n,0.002341,0.000000,0.005682,0.005682,0.004185,0.005682,0.001044,0.007646,0.000654,0.000654,...,0.000475,0.001551,0.000858,0.001155,0.000957,0.001724,0.001875,0.002058,0.002221,0.002937
"AT1G01100.1 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n",0.004177,0.005682,0.000000,0.000000,0.002294,0.000000,0.006351,0.003700,0.005692,0.005692,...,0.005975,0.006996,0.006069,0.004850,0.005593,0.006989,0.004748,0.004317,0.007330,0.004736
"AT1G01100.2 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n",0.004177,0.005682,0.000000,0.000000,0.002294,0.000000,0.006351,0.003700,0.005692,0.005692,...,0.005975,0.006996,0.006069,0.004850,0.005593,0.006989,0.004748,0.004317,0.007330,0.004736
"AT1G01100.3 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=96\n",0.002867,0.004185,0.002294,0.002294,0.000000,0.002294,0.004750,0.005542,0.004505,0.004505,...,0.004627,0.005569,0.004814,0.003575,0.004459,0.005693,0.003569,0.002848,0.005468,0.003333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATMG00980.1 | Symbols: RPSL2 | | chrm:288446-288823 FORWARD LENGTH=125\n,0.003497,0.001724,0.006989,0.006989,0.005693,0.006989,0.001605,0.008705,0.001554,0.001554,...,0.001628,0.000821,0.001534,0.002345,0.001897,0.000000,0.003047,0.003319,0.001342,0.004269
ATMG00990.1 | Symbols: NAD3 | 

In [ ]:
np.save("wesserstein_distance.npy", wessertein_dist)

In [ ]:
df.to_csv('wassertein.csv', index=False)

In [ ]:
column_names = df.columns

In [ ]:
np.save("column_names.npy", column_names)